## Setup

In [95]:
from evaluate import *
import pandas as pd
from sklearn.model_selection import LeaveOneOut
import warnings
from collections import Counter, defaultdict
from itertools import combinations
import os
import pickle

In [96]:
data_root = r'RSRM\data\SR\\'

In [97]:
dataset='s'
regression = True if dataset == 's' else False
cv_info = f'Treated results/{dataset}_folds.txt'
linear_cv_data = f'Treated results/linear_{dataset}_cross_validation.csv'
pysr_cv_data = f'Treated results/pysr_{dataset}_cross_validation.csv'
rsrm_cv_data = f'Treated results/rsrm_{dataset}_cross_validation.csv'
data_file = f'{dataset}'
metric = lambda t, mu: 1 - poisson_deviance(t, mu) / np.sum( t * np.log( (t + 1e-10) / np.mean(t)) )

In [98]:
folds = read_train_test_indices(cv_info)
dataset_name = {'lepthyphantes': 'canariphantes', 'alestrus': 'alestrus', 's': 'species richness'}[dataset]
dataset_name = dataset_name.capitalize()
if regression:
    metric = 'Pseudo R²'
else:
    metric = 'TSS'
linear_df_o = pd.read_csv(linear_cv_data)
pysr_df_o = pd.read_csv(pysr_cv_data)
rsrm_df_o = pd.read_csv(rsrm_cv_data)
X1, y1 = load_dataset(data_root + data_file + '_train.csv')
X2, y2 = load_dataset(data_root + data_file + '_test.csv')
X = pd.concat([X1, X2], ignore_index=True) 
y = pd.concat([y1, y2], ignore_index=True).astype(int)
get_fold = get_fold_wrapper(X, y, folds)
originals = {'linear': linear_df_o, 'pysr': pysr_df_o, 'rsrm': rsrm_df_o}

In [99]:
linear_df = pd.DataFrame(columns=['run', 'equation', 'complexity', 'train', 'test'])
pysr_df = pd.DataFrame(columns=['run', 'equation', 'complexity', 'train', 'test'])
rsrm_df = pd.DataFrame(columns=['run', 'equation', 'complexity', 'train', 'test'])
new_ones = {'linear': linear_df, 'pysr': pysr_df, 'rsrm': rsrm_df}
predictions = {}
for key in originals.keys():
    new_df = new_ones[key]
    original_df = originals[key]
    new_df['skeleton'] = original_df['Equation'].apply(replace_numeric_parameters)
    new_df['run'] = original_df['Run']
    new_df['complexity'] = original_df['Equation'].apply(count_weighted_operations)
    new_df.drop(new_df[new_df['complexity'] == 0].index, inplace=True)
    new_df.drop(new_df[new_df['complexity'] > 20].index, inplace=True)
    new_df['equation'] = original_df['Equation']

# Experiments

In [ ]:
# Try it with (exp, linear): (C = C, C = A, C = 1)
# C is a fittable parameter whereas A is a shape parameter and so is fixed, a constant of nature of sorts
class Parameter:
    def __init__(self, initial_value):
        self.value = initial_value
        self.fitted = None
        self.parameter_set = {1} # Maybe make a subclass with different parameter sets properties/functions but only if it has been fitted
        pass
class FeatureRepr:
    def __init__(self, var_name):
        self.var = var_name
        self.initial_value_C = [1]
        # self.all_fitted_C = []   # All fitted parameters associated to feature, X**C or exp(C*X)
        self.final_C = None
        self.A = None   # Fitted A
    def add_to(self, expression):
        pass
    def log_str(self):
        pass
    def update_initial_value(self, C):
        self.initial_value_C = [C]

class PolyFeatRepr(FeatureRepr):
    def add_to(self, expression):
        expression.add_poly(self)
    def __repr__(self):
        return f'{self.var}**{self.final_C}'
    def __str__(self):
        return f'{self.var}**C'
    def log_str(self):
        return f'C*log({self.var})'

class ExpFeatRepr(FeatureRepr):
    def add_to(self, expression):
        expression.add_exp(self)
    def __repr__(self):
        return f'{self.final_C}**{self.var}'
    def __str__(self):
        return f'exp(C*{self.var})'
    def log_str(self):
        return f'C*{self.var}'

class LogFeatRepr(FeatureRepr):
    def __init__(self, var_name):
        super().__init__(var_name)
        self.initial_value_C = [0,1]
    def add_to(self, expression):
        expression.add_log(self)
    def __repr__(self):
        return f'({self.final_C[0]} + log({self.var}))**{self.final_C[1]}'
    def __str__(self):
        return f'(C+log({self.var}))**C'
    def log_str(self):
        return f'C*log(log({self.var}))'
    def update_initial_value(self, C):
        self.initial_value_C = [0, C]

class ExpressionRepr:
    def __init__(self, feature_list):
        self.expr = {'exp': [], 'poly': [], 'log': []}
        for feat in feature_list:
            feat.add_to(self)            
    def __repr__(self):
        return self.basis_function()
    def add_poly(self, feat):
        self.expr['poly'].append(feat)
    def add_exp(self, feat):
        self.expr['exp'].append(feat)
    def add_log(self, feat):
        self.expr['log'].append(feat)
    def basis_function(self):
        poly_basis = ' * '.join([str(i) for i in self.expr['poly']])
        exp_basis = ' * '.join([str(i) for i in self.expr['exp']])
        log_basis = ' * '.join([str(i) for i in self.expr['log']])
        non_empty_basiss = [i for i in [poly_basis, exp_basis, log_basis] if i]
        basis = ' * '.join(non_empty_basiss)
        return basis
    def log_basis_function(self):
        poly_basis = ' * '.join([str(i) for i in self.expr['poly']])
        exp_basis = ' * '.join([str(i) for i in self.expr['exp']])
        log_basis = ' * '.join([str(i) for i in self.expr['log']])
        non_empty_basiss = [i for i in [poly_basis, exp_basis, log_basis] if i]
        basis = ' * '.join(non_empty_basiss)
        return basis

In [185]:
from itertools import combinations
from copy import deepcopy
basinhop = True
variables = X.columns.tolist()
res2 = []
all_single_features = [PolyFeatRepr(var) for var in variables] + [ExpFeatRepr(var) for var in variables] + [LogFeatRepr(var) for var in variables]
all_feature_combinations = [feature_comb for num_feats in range(1,4) for feature_comb in combinations(all_single_features, num_feats)]
all_feature_combinations = [[deepcopy(feat) for feat in feature_comb] for feature_comb in all_feature_combinations]
all_expressions = [ExpressionRepr(feature_comb) for feature_comb in all_feature_combinations]
all_expressions

[H**C,
 Sl**C,
 P**C,
 T**C,
 D**C,
 exp(C*H),
 exp(C*Sl),
 exp(C*P),
 exp(C*T),
 exp(C*D),
 (C+log(H))**C,
 (C+log(Sl))**C,
 (C+log(P))**C,
 (C+log(T))**C,
 (C+log(D))**C,
 H**C * Sl**C,
 H**C * P**C,
 H**C * T**C,
 H**C * D**C,
 H**C * exp(C*H),
 H**C * exp(C*Sl),
 H**C * exp(C*P),
 H**C * exp(C*T),
 H**C * exp(C*D),
 H**C * (C+log(H))**C,
 H**C * (C+log(Sl))**C,
 H**C * (C+log(P))**C,
 H**C * (C+log(T))**C,
 H**C * (C+log(D))**C,
 Sl**C * P**C,
 Sl**C * T**C,
 Sl**C * D**C,
 Sl**C * exp(C*H),
 Sl**C * exp(C*Sl),
 Sl**C * exp(C*P),
 Sl**C * exp(C*T),
 Sl**C * exp(C*D),
 Sl**C * (C+log(H))**C,
 Sl**C * (C+log(Sl))**C,
 Sl**C * (C+log(P))**C,
 Sl**C * (C+log(T))**C,
 Sl**C * (C+log(D))**C,
 P**C * T**C,
 P**C * D**C,
 P**C * exp(C*H),
 P**C * exp(C*Sl),
 P**C * exp(C*P),
 P**C * exp(C*T),
 P**C * exp(C*D),
 P**C * (C+log(H))**C,
 P**C * (C+log(Sl))**C,
 P**C * (C+log(P))**C,
 P**C * (C+log(T))**C,
 P**C * (C+log(D))**C,
 T**C * D**C,
 T**C * exp(C*H),
 T**C * exp(C*Sl),
 T**C * exp(C*P

In [183]:
for expr in all_feature_combinations:
    num_feats = len(expr)
    basis = ' * '.join([f'{i}**C' for i in expr])
    log_eq = 'C + ' + '+ '.join([f'C * log({var})' for var in expr])
    Cs, _ = optimize_equation(log_eq, normalize_by_iqr(X), np.log(y + 1), variables, loss='mse', initial_guess=np.ones(num_feats+1), basinhop=False)
    Cs = Cs.tolist()
    eq_intrcp = f'C + C * {basis}'
    Cs_intrcp, _ = optimize_equation(eq_intrcp, normalize_by_iqr(X), y, variables, loss='poisson', initial_guess=[-1, np.exp(Cs[0])] + Cs[1:], basinhop=False, minimizer_kwargs = {"method": "Powell"})
    # eq_linear2 = f'C + {basis}'
    # Cs2, _ = optimize_equation(eq_linear2, normalize_by_iqr(X), y, variables, loss='poisson', initial_guess=[1, Cs[0]], basinhop=False, minimizer_kwargs = {"method": "Powell"})
    eq_no_intrcp = f'C * {basis}'
    Cs_no_intrcp, _ = optimize_equation(eq_no_intrcp, normalize_by_iqr(X), y, variables, loss='poisson', initial_guess=[np.exp(Cs[0])] + Cs[1:], basinhop=False, minimizer_kwargs = {"method": "Powell"})
    
    eqs_linear = [eq_intrcp, eq_no_intrcp] 
    Cs = [Cs_intrcp, Cs_no_intrcp]
    all_metrics = []
    for eq_linear, params in zip(eqs_linear, Cs):
            equation_str_fitted = replace_optimized_parameters(eq_linear, params)
            eq = PoissonEquation(equation_str_fitted, variables)
            _, y_pred, all_params, _ = loocv(eq, X, y)
            v = metric(y, y_pred)
            print(equation_str_fitted)
            all_last_params = [params['Cs'][-1] for params in all_params]
            print(f'Mean: {np.mean(all_last_params)}, Min: {np.min(all_last_params)}, Max: {np.max(all_last_params)}, Range: {np.max(all_last_params) - np.min(all_last_params)}')
            print(v)
            all_metrics.append(v)
    # Find the index of the smallest loss
    best_index = all_metrics.index(max(all_metrics))

    # Get the corresponding C values and minimum loss
    best_eq_linear = eqs_linear[best_index]
    best_Cs = Cs[best_index]
    best_eq = replace_optimized_parameters(best_eq_linear, best_Cs)
    best_metric = all_metrics[best_index]
    res2 += [(best_eq, best_metric)]

for best_eq, best_metric in res2:
      print(f'Equation: {best_eq}, Best: {best_metric}')

TypeError: _lambdifygenerated() missing 1 required positional argument: 'C2'

Try to fit x^k, with k = \{n/2\} U \{.25, 1/3, 2/3, 4/3\}, for the 3 different ks: k=1 and the 2 k closest to the mean of the parameter fit on the whole dataset. For C^x it doesn't make sense (x is normalized anyway and integer powers don't make sense). Do it for log(x)^C as well.

Next steps:

0. ~~Implement basic functionality~~
1. Use BFGS and jacobian w.r.t. Cs for it to be faster
2. After fitting the loocv with all parameters do the custom rounding using the information learned from loocv and run it again without fitting the rounded parameters
3. Include more variables in the current basis function
4. Alter step 2. to do custom rounding for {}, {var1}, {var2}, ..., {var1, var2}, ... 
5. Include more basis functions (log and exp)
6. Include denominator basis function
7. Combine 5. and 6.. To prune the search select only ones with best $R^2$ for the next step
8. Combine linearly best learned basis functions and evaluate them with LOOCV then yielding a HOF that will surely crush PySR and RSRM

In [43]:
rsrm_df

,run,equation,complexity,train,test,skeleton,loo_r2
0,1,2.22575163521395**H,3,0.248446,0.193267,C**H,0.440199
1,1,(H + 1.5358194749129181)/D,4,0.300417,0.254234,(H + C)/D,0.541995
2,1,(H + 15.260619115027552)/(D*T),5,0.302567,0.242438,(H + C)/(D*T),0.542808
3,1,-1.2832712494830179*D*H - H + P + exp(H),8,0.302024,0.225241,C*D*H - H + P + exp(H),0.325983
4,2,0**T,2,-26.632335,-23.241700,C**T,-50.120190
5,2,3.616659651492994/D,3,0.260948,0.236755,C/D,0.491570
6,2,0.35348059496783707*exp(H),3,0.266456,0.176268,C*exp(H),0.485240
7,2,(H + 1.530966402021697)/D,4,0.291786,0.223560,(H + C)/D,0.541995
8,2,exp(2*H - P),4,0.296538,0.175398,exp(C*H - P),0.515349
9,2,4.759341124102596**H/P**2,7,0.303841,0.201917,C**H/P**C,0.517117


In [19]:
import sympy
# str(sympy.simplify(sympy.expand("(X+C)**3*(X-C) + log(4*X^2*(exp(2+X)))")))
# str(sympy.expand("X1 + 183.733256531987*(0.0737744726316052*X1 + 1)**2"))
# str(sympy.expand("log(C*X**2)"))
# str(sympy.expand("exp(C*X**2 - Y)"))
# str(sympy.expand_trig("tanh(C*X**2 - Y)"))
str(sympy.expand("(X-4)/(Y+5)"))

'X/(Y + 5) - 4/(Y + 5)'